In [1]:
import requests
import time
import
from google.colab import drive
import os

In [ ]:
# Mounting drive and changing working directory to Milestone raw folder, so data will be written there
drive.mount('/content/drive')
path = '/content/drive/My Drive/Milestone II - T31/Code/data/raw'
os.chdir(path)

In [2]:
def get_access_token(client_id, client_secret):
    """
    Obtain an access token using client credentials.
    """
    url = 'https://sandbox.bcda.cms.gov/auth/token'
    headers = {
        'accept': 'application/json'
    }
    response = requests.post(url, headers=headers, auth=(client_id, client_secret))
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("Access token obtained successfully.")
        return access_token
    else:
        print(f"Failed to get access token: {response.status_code}, {response.text}")
        return None

In [3]:
def start_job(access_token):
    """
    Start a data export job.
    """
    url = 'https://sandbox.bcda.cms.gov/api/v2/Group/all/$export'
    headers = {
        'accept': 'application/fhir+json',
        'Prefer': 'respond-async',
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 202:
        job_url = response.headers.get('Content-Location')
        print(f"Job started successfully. Job URL: {job_url}")
        return job_url
    else:
        print(f"Failed to start job: {response.status_code}, {response.text}")
        return None

In [4]:
def check_job_status(job_url, access_token):
    """
    Check the status of the data export job until it is complete.
    """
    headers = {
        'accept': 'application/fhir+json',
        'Authorization': f'Bearer {access_token}'
    }
    while True:
        response = requests.get(job_url, headers=headers)
        if response.status_code == 202:
            # Job is still processing
            print("Job is still processing... Checking again in 60 seconds.")
            time.sleep(60)  # Wait before checking again
        elif response.status_code == 200:
            # Job is done
            print("Job completed successfully.")
            return response.json()
        else:
            print(f"Failed to check job status: {response.status_code}, {response.text}")
            return None

In [5]:
def download_data_files(output_list, access_token):
    """
    Download the data files from the job output.
    """
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept-Encoding': 'gzip'
    }
    for item in output_list:
        file_type = item.get('type')
        url = item.get('url')
        print(f"Downloading {file_type} data from {url}...")
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            filename = f"{file_type}.ndjson"
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Saved {filename}")
        else:
            print(f"Failed to download {file_type}: {response.status_code}, {response.text}")

In [6]:
def main():
    access_token = get_access_token(client_id, client_secret)
    if not access_token:
        return
    job_url = start_job(access_token)
    if not job_url:
        return
    job_result = check_job_status(job_url, access_token)
    if not job_result:
        return
    output = job_result.get('output')
    if not output:
        print("No output data available.")
        return
    download_data_files(output, access_token)

In [16]:
# Public Credentials obtained from the Partially Adjudicated Claims Datasets (Large REACH ACO)

client_id = '2121efbd-98d2-4323-84db-974c8864abc7'
client_secret = '6eed5c5dd69422ca0f7cb0c4912e3e06c3cd043f2d22d71eee22ea224285e4b9e74667c0de004034'

main()

Access token obtained successfully.
Job started successfully. Job URL: https://sandbox.bcda.cms.gov/api/v2/jobs/71805
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job is still processing... Checking again in 10 seconds.
Job completed successfully.
Saved Claim.ndjson
Saved Claim.ndjson
Saved Claim.ndjson
Saved ClaimResponse.ndjson
Saved ClaimResponse.ndjson
Saved ClaimResponse.ndjson


In [8]:
# Public Credentials obtained from the Large ADvanced Model Entity dataset
client_id = '0a0c75f0-da95-4198-9c0f-666b41e21017'
client_secret = '4c1145e6e294ec6852c0bf1c24b38c7f5af99a8a69423616ebb42af84db74fc903aad99b1632bf51'

main()